## Web App

The objective of this project is to create two dockerfiles for a web application in nodejs.

1. The first dockerfile will allow to create an image to develop code (development environment, presence of hotreload).

2. The second dockerfile will create a compiled and optimised version of the application ready to be deployed in production with nginx.

This is a difficult exercise. You can use the help and correction if you are stuck. Do not hesitate to reread the course. A todolist application is provided.

This project is deliberately difficult and formative. It is a concrete subject.
With the help of the detailed correction you can help the students on the blocking points.

## Development mode

Dockerfile

In [ ]:
FROM node:10.18.1
WORKDIR /app
COPY package.json ./
RUN npm i
CMD npm run serve

1. Node allows you to compile the application in real time for hot reload.

2. Create a folder in the image to put the dependencies of the application. Be careful if you add a dependency during the development of the application, you will have to think about rebuilding the image.

3. Copy of the .json package and installation of the dependencies

4. The run command corresponds to the command issued when the container is started. serve allows hot reload to be used.

Application build

In [ ]:
%%bash
docker build -t todo:dev -f Dockerfile.dev .

1. The -f option allows to choose a specific dockerfile

Container instanciation

In [ ]:
%%bash
docker container run -v ${PWD}:/app -v /app/node_modules -p 8080:8080 todo:dev

After building this image with the dependencies you need to instantiate a container with a binding of the source folder in the same folder as the dependencies.

1. We will bind the current folder in the container, the sources can be modified.

2. We use an anonymous volume to avoid overwriting module nodes during source linking 

3. The server launched by the npm command listens on port 8080

## Production mode

In [ ]:
# BUILD
FROM node:10.18.1 as build-stage
WORKDIR /app
COPY . .
RUN npm i
RUN npm run build

# PROD
FROM nginx:stable-alpine as production-stage
COPY --from=build-stage /app/dist /var/www
COPY nginx.conf /etc/nginx/nginx.conf
EXPOSE 80
CMD ["nginx", "-g", "daemon off;"]

1. Node will allow the construction of the application.

2. Copy of sources, installation of dependencies and construction of the application.

The source code is now no longer useful we want to keep only the transcompiled files.

3. Let's use the nginx image to serve these files. Just add the configuration file to replace the default one. 

4. Copy of the application.

5. To the WWW file served by nginx

6. The server will be started when the container is instantiated.

Application build

In [ ]:
%%bash
docker image build -t todo:prod .

Starting the container

In [ ]:
%%bash
docker container run -p 8080:80 todo:prod